# Multilingual Search


**Using Multilingual embedding model search is performed to answer questions in any language.**


**Flow of multilingual search:**

**1. Data preparation: Loading and chuncking data**

**2. Embeddings: EMbed data using multilingual embedding model**

**3. Vector store: Add data to FAISS vector store**

**4. Retrieval: Extract relevant documents from vetor store for query in any language (depending on embedding model supporting languages)**

**5. Answer generation: Generate answer from retrieved context**

## Import Libraries

In [1]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

C:\Users\PoojaT.WINJITBIOS\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load and chunk PDF data

**The data loaded is in english language.**

In [2]:
# loading pdf data and chunking

pdf_data = PDFPlumberLoader(r"D:\Winjit_training\R&D\MM-LLM\2306.13549v2.pdf").load()
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=50,
)
data_splits = splitter.split_documents(pdf_data)
print("no of chunks: ", len(data_splits))


no of chunks:  45


## Multilingual Embedding

In [3]:
# multilingual huggingface embedding model
model_name = "Alibaba-NLP/gte-multilingual-base"
embeddings = HuggingFaceEmbeddings(
                model_name=model_name,
                model_kwargs={"device": "cpu", "trust_remote_code": True},
                encode_kwargs={"normalize_embeddings": True},
            )

C:\Users\PoojaT.WINJITBIOS\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactl

## Add documents to vector store

In [4]:
# faiss vectorstore
vector_store = FAISS(
    embedding_function=embeddings,
    index=faiss.IndexFlatL2(768),
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
# print(vector_store)
vector_store.add_documents(documents=data_splits)

['27012f16-0c59-43f9-8ff5-78672d1c432f',
 '52d24163-aadb-4ed2-8197-dc656841ef6f',
 'e328a6a7-2ac2-4576-adca-041b08a34bf9',
 '0a5fcd1d-9079-4680-b600-773e246ed1aa',
 '9d8a1d88-6eba-4ddd-964b-97609135b122',
 'ff1f5b98-1f8d-49ee-958d-b921977ae42a',
 '4e9971bb-d9e2-4a4c-b615-4c0256bd7580',
 'ce8afadd-2c31-43c2-aa0c-a6bf160ccb39',
 '30c218b8-b1d2-47a6-bdb3-f193bee8dc3c',
 '1fd38ade-3694-4089-8deb-416c12345228',
 'db19986c-b1f5-47f0-b6fb-e46fdfdbaa77',
 '8b10a599-fae9-429f-b2dd-a6c77a727f69',
 'eb643dbd-cee6-47e5-b787-04c6d3223940',
 '32d6fd38-52a6-4822-897a-33aa79f1f8bb',
 '92ea899b-9e40-4561-911e-67d9c789bc50',
 'd1888dfc-f801-4b1c-8804-0e3f26a4ddb5',
 'a088019b-fd40-42bb-a2bd-e2792aa68cc5',
 '33f95767-6537-44a4-90c0-470fa7eccdcb',
 '371ab4d6-6cf8-4170-adcd-6744aedb8d7f',
 '43a0d842-c9b7-4df5-968c-8a499b7cacb9',
 '62fa7568-2e3f-482e-a00a-8b64e709867d',
 '2512d7be-41d4-4440-bf71-359d5f6d86a2',
 '4b1f9a9c-6055-4d97-a4ce-f9c20dff48c2',
 '6db03560-e4a7-4be8-bfbb-e7a12249a613',
 '09a5cb60-407f-

## Retriever

In [5]:
# multilingual retriever
multilingual_retriever = vector_store.as_retriever()    

## Google LLM

In [6]:
# define LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key="Google API KEY")

## Multilingual QA

In [13]:
# non english query
query = "Was ist ein Modalitätsencoder?"

# retrieve documents
out_docs = multilingual_retriever.invoke(query)
context = "\n--\n".join(doc.page_content for doc in out_docs)

# Answer generation 
prompt_1 = PromptTemplate.from_template("You are a multilingual assistant. Answer user's {query} from given {context} in same language as query.")
chain = prompt_1 | llm
response = chain.invoke({'query': query,
                            'context': context})

In [14]:
print("query: \n",query)
print("response: \n", response.content)

query: 
 Was ist ein Modalitätsencoder?
response: 
 Ein Modalitäts-Encoder ist ein Teil eines multimodalen Sprachmodells (MLLM), der rohe Informationen aus verschiedenen Modalitäten, wie Bilder, Audio oder Videos, in eine kompaktere Darstellung umwandelt. Diese Darstellung kann dann von einem großen Sprachmodell (LLM) verarbeitet werden, das normalerweise auf Textdaten trainiert wurde.

**Funktionsweise:**

* **Komprimierung:** Der Encoder komprimiert die rohen Informationen, z. B. ein Bild, in eine kleinere Anzahl von Merkmalen (Features).
* **Repräsentation:** Die resultierenden Features repräsentieren die wichtigsten Informationen aus der ursprünglichen Modalität in einer Weise, die für das LLM verständlich ist.

**Beispiele:**

* **Bild-Encoder:** Ein Bild-Encoder könnte ein Bild in eine Reihe von Vektoren umwandeln, die die Farben, Formen und Texturen des Bildes darstellen.
* **Audio-Encoder:** Ein Audio-Encoder könnte ein Audiosignal in eine Reihe von Vektoren umwandeln, die die 